In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data# this contains our dataset
import numpy as np
from torch_geometric.datasets import Planetoid# this helps us manipulate the dataset
from torch_geometric.nn import GCNConv, SAGEConv, GATConv# this is a simple implementation of the GCN convolutional layer
import torch_geometric.transforms as T
from torch_geometric.utils import to_dense_adj
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
# from torch_geometric.nn import Data
# ipyvolume
# maybe use global mean pool

In [50]:
class GCN(torch.nn.Module):# this is the torch geometric implementation of our GCN model like before, it
    # is a lot simpler to implement and way customizeable
    def __init__(self, in_feat, hid_feat, out_feat):
        super().__init__()
        self.conv1 = GCNConv(in_feat, hid_feat)
        self.final = GCNConv(hid_feat, out_feat)
        self.activation = nn.ReLU()
        #self.dropout = nn.Dropout(p=.4)

    def forward(self, feats, edge_index,batch   ):
        x = feats
        x1 = self.activation(self.conv1(x, edge_index))

        # Maybe get rid of dropout
        x2 = F.dropout(x1, training= self.training)

        x7 = self.final(x2, edge_index)
        return global_mean_pool(x7,batch)

In [51]:
criterion = nn.MSELoss()
num_feats = 10 #FIXME
num_hid = 16
epochs = 30
output = 1
model = GCN(num_feats,num_hid,output).to("cuda:0")# this initializes our model and puts it on the GPU(change if you do not have a gpu)
optimizer = torch.optim.Adam(params=model.parameters(),lr = 5e-3, weight_decay=5e-4)# sets up our optimizer
train_feats = None # fill this with a torch Tensor (num_graphs x num_nodes x num_feats)
train_edge_index = None # fill this with a torch Tensor (num_graphs x num_edges x 2)
train_labels = None # fill this with a torch Tensor (num_graphs x 1)

val_feats = None # fill this with a torch Tensor
val_edge_index = None # fill this with a torch Tensor
val_labels = None # fill this with a torch Tensor

test_feats = None # fill this with a torch Tensor
test_edge_index = None # fill this with a torch Tensor
test_labels = None # fill this with a torch Tensor

# For dataset preprocessing, for each dataset do the following
#   Create a data object for each graph Data(x=x,edge_index=edge_index,y=labels)
#   make a list of those objects [data1,data2,...]
#   loader = DataLoader(datasetlist, batch_size=64)

In [52]:
def accuracy(preds, labels):
    return torch.mean(torch.abs(preds - labels))

In [81]:
edge_index = torch.tensor([[0, 1, 2,3],[1,2,0,2]]).long().cpu()
print(type(edge_index))
feat = torch.arange(10).tolist()
feats = torch.Tensor([feat,feat,feat,feat]).cpu()
model = model.cpu()
a = (model(feats,edge_index,torch.Tensor([0,0,0,0]).long()))
data1 = Data(x=feats,edge_index=edge_index,y=torch.Tensor([[1]]))
data2 = Data(x=feats,edge_index=edge_index,y=torch.Tensor([[0]]))
datalist = [data1,data2]
datalist2 = [data2,data1]
# dataset = torch.utils.data.TensorDataset(datalist,datalist2)
Loader = DataLoader(datalist,batch_size=1)
Loader2 = DataLoader(datalist2,batch_size=1)
loader2_iterator = iter(Loader2)
for d1 in Loader:
    try:
        d2 = next(loader2_iterator)
    except StopIteration:
        loader2_iterator = iter(Loader2)
        d2 = next(loader2_iterator)
    
    print(d1.y,d2.y)
# print(a)
# accuracy(a,torch.Tensor([[0],[0]]))

<class 'torch.Tensor'>
tensor([[1.]]) tensor([[0.]])
tensor([[0.]]) tensor([[1.]])


In [55]:
model.train()# tells our model we are about to train
val_accs = []
train_accs = []

train_losses = []
val_losses = []

test_losses = []
test_accs = []
# for epoch in range(100):# runs through all the data 200 times
#     optimizer.zero_grad()
#     train_preds = model(train_feats, train_edge_index)# runs our model on the data

#     train_loss = criterion(train_preds, train_labels)
#     train_acc = accuracy(train_preds, train_labels)

#     val_preds = model(val_feats, val_edge_index)

#     val_loss = criterion(val_preds, val_edge_index)
#     val_acc = accuracy(val_preds, val_edge_index)

#     if(epoch %5 == 0):
#         print(f"epoch: {epoch} training_loss: {train_loss:.2} train_abs_error: {train_acc:.2} val_loss: {val_loss:.2} val_abs_error: {val_acc:.2}")
        
#     train_loss.backward()
#     optimizer.step()
#     val_accs.append(val_acc)
#     train_accs.append(train_acc)

#     train_losses.append(train_loss.cpu().item())
#     val_losses.append(val_loss.cpu().item())
def train():
    model.train()
    val_iterator = iter(val_loader)
    for i,train_data in enumerate(train_loader):
        try:
            val_data = next(val_iterator)
        except StopIteration:
            val_iterator = iter(val_loader)
            val_data = next(val_iterator)

        train_out = model(train_data.x,train_data.edge_index,train_data.batch)
        train_loss = criterion(train_out,train_data.y)
        train_absolute_error = accuracy(train_out,train_data.y).cpu().item()
        print(f"Mean squared error for training batch {i} is: {train_loss.item(),:.2}, absolute error is: {train_absolute_error:.2}")
        train_losses.append(train_loss.cpu().item())
        train_accs.append(train_absolute_error)

        val_out = model(val_data.x,val_data.edge_index,val_data.batch)
        val_loss = criterion(val_out,val_data.y)
        val_absolute_error = accuracy(val_out,val_data.y).cpu().item()
        print(f"Mean squared error for validation batch {i} is: {val_loss.item(),:.2}, absolute error is: {val_absolute_error:.2}")
        val_losses.append(val_loss.cpu().item())
        val_accs.append(val_absolute_error)

        train_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

def test():
    model.eval()
    for i,data in enumerate(test_loader):
        out = model(data.x,data.edge_index,data.batch)
        loss = criterion(out,data.y)
        absolute_error = accuracy(out,data.y).cpu().item()
        print(f"Mean squared error for testing batch {i} is: {loss.item(),:.2} absolute error is: {absolute_error:.2}")
        test_losses.append(loss.cpu().item())
        test_accs.append(absolute_error)

for epoch in epochs:
    print(f"epoch: {epoch}")
    train()
    test()


In [ ]:
df_accs = pd.DataFrame({"train_acc":train_accs,"val_acc":val_accs})
sns.relplot(data=df_accs, kind = "line")
df_losses = pd.DataFrame({"train_loss":train_losses,"val_loss":val_losses})
sns.relplot(data=df_accs, kind = "line")
plt.show()